<a href="https://colab.research.google.com/github/CrisMcode111/DI_Bootcamp/blob/main/w9_d2_Daily_Challenge_MCP_Airbnb_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student Notebook - MCP + Airbnb (Colab)

Reference notebook: local notes MCP + Airbnb MCP + optional real LLM.

## Install
Run once. npm only needed for the real Airbnb server.

### Environment Setup Notes

The following cells install and configure all dependencies required for the MCP client/server workflow:
- `mcp`, `nest_asyncio`, and `requests` for the local MCP client execution inside Colab.
- `azure-ai-inference` for optional LLM integration.
- The Airbnb MCP Server (`@openbnb/mcp-server-airbnb`) is installed globally via npm, enabling a real server backend when available.

Because Google Colab does not provide traditional TTY file descriptors, the `OutStream.fileno()` method is patched so the MCP server spawned over STDIO can communicate correctly with the client. This workaround is required only in notebook environments.

We also load environment variables:
- `MCP_HTTP_TOKEN` used for server authentication.
- `GITHUB_TOKEN` needed when enabling a real LLM backend.

Up to this point, all setup commands executed successfully, meaning the environment is ready for running the MCP client and interacting with tools.


In [1]:
!pip install -q mcp nest_asyncio requests
!pip install azure-ai-inference

# Optional: real Airbnb server
!npm install -g @openbnb/mcp-server-airbnb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.3/213.3 kB 14.3 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴npm warn deprecated node-domexception@1.0.0: Use your platform's native DOMException instead
⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 117 packages in 12s
⠧
⠧42 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [2]:
import sys
from ipykernel.iostream import OutStream

def _patched_fileno(self):
    # stdout → 1, stderr → 2
    if self is sys.stderr:
        return 2
    return 1

# Patch the class for all OutStream instances
OutStream.fileno = _patched_fileno

# And patch the current instances explicitly
sys.stdout.fileno = lambda: 1
sys.stderr.fileno = lambda: 2


## Config
Flip toggles as needed. Keep defaults for stubbed run.

In [3]:

import os
from pathlib import Path

MCP_HTTP_TOKEN = os.getenv("MCP_HTTP_TOKEN", "devtoken123")
USE_REAL_AIRBNB = True  # True if npm server available
USE_REAL_LLM = True     # True if GITHUB_TOKEN set


In [5]:
import os
BASE_ENV = os.environ.copy()
BASE_ENV["MCP_HTTP_TOKEN"] = MCP_HTTP_TOKEN


In [6]:
import os
from google.colab import userdata  # Colab secrets API

# If your secret is saved under the key "GITHUB_TOKEN" in Colab:
os.environ["GITHUB_TOKEN"] = userdata.get("GITHUB_TOKEN")

# If you used a different key name in the secrets UI, e.g. "github_token":
# os.environ["GITHUB_TOKEN"] = userdata.get("github_token")

print("GITHUB_TOKEN visible to Python:", bool(os.getenv("GITHUB_TOKEN")))


GITHUB_TOKEN visible to Python: True


## Local notes MCP server

In [8]:
LOCAL_SERVER = Path("local_notes_server.py")
LOCAL_SERVER.write_text(
'''from mcp.server.fastmcp import FastMCP

notes = []

# Construct FastMCP server instance
mcp = FastMCP("local-notes-server")

# Register add_note as a tool
@mcp.tool()
def add_note(text: str) -> str:
    "Add a note to the in-memory list."
    notes.append(text)
    return f"Saved note #{len(notes)}: {text}"

# Register list_notes as a tool
@mcp.tool()
def list_notes() -> str:
    "List saved notes."
    if not notes:
        return "No notes yet"
    return "\\n".join(f"{i+1}. {n}" for i, n in enumerate(notes))

if __name__ == "__main__":
    mcp.run()
''',
    encoding="utf-8",
)
print("wrote", LOCAL_SERVER)


wrote local_notes_server.py


## Client helpers (convert tools, stub planner, optional real LLM)

In [9]:

import asyncio
import json
import nest_asyncio
from typing import Any, Dict, List
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

nest_asyncio.apply()

def convert_tool(tool, prefix: str):
    # Azure requires ^[a-zA-Z0-9_\.-]+$, so no slashes
    fn_name = f"{prefix}__{tool.name}"
    return {
        "type": "function",
        "function": {
            "name": fn_name,
            "description": tool.description or "mcp tool",
            "parameters": {
                "type": "object",
                "properties": tool.inputSchema.get("properties", {}),
                "required": tool.inputSchema.get("required", []),
            },
        },
    }



def call_llm(prompt: str, functions: List[Dict[str, Any]], use_real: bool = True):
    import os
    import json
    from azure.ai.inference import ChatCompletionsClient
    from azure.core.credentials import AzureKeyCredential

    token = os.getenv("GITHUB_TOKEN")
    if not token:
        raise RuntimeError("Set GITHUB_TOKEN or use stub planner.")

    client = ChatCompletionsClient(
        endpoint="https://models.inference.ai.azure.com",
        credential=AzureKeyCredential(token)
    )

    # Azure Chat Completion (OpenAI-compatible)
    resp = client.complete(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        tools=functions,
        tool_choice="auto"
    )

    calls = []
    msg = resp.choices[0].message

    for tc in msg.tool_calls or []:
        args = tc.function.arguments
        args_json = json.loads(args) if isinstance(args, str) else args
        calls.append({
            "name": tc.function.name,
            "args": args_json
        })

    return calls


In [10]:
def answer_with_llm(
    user_prompt: str,
    tool_calls: List[Dict[str, Any]],
    tool_results: List[Dict[str, Any]],
    use_real: bool = True,
) -> str:
    import os
    import json

    # MINIMAL FIX: shrink tool_results before sending to gpt-4o
    small_results = []
    for r in tool_results:
        content = r.get("content", [])
        short_content = []
        if content:
            first = content[0]
            if isinstance(first, str) and len(first) > 4000:
                first = first[:4000] + "...(truncated)..."
            short_content = [first]
        small_results.append(
            {
                "name": r.get("name"),
                "args": r.get("args", {}),
                "content": short_content,
            }
        )


    from azure.ai.inference import ChatCompletionsClient
    from azure.core.credentials import AzureKeyCredential

    token = os.getenv("GITHUB_TOKEN")
    if not token:
        raise RuntimeError("Set GITHUB_TOKEN or use_real=False in answer_with_llm.")

    client = ChatCompletionsClient(
        "https://models.inference.ai.azure.com",
        AzureKeyCredential(token),
    )

    payload = {
        "user_question": user_prompt,
        "tool_calls": tool_calls,
        # use the shrunk version here
        "tool_results": small_results,
    }

    resp = client.complete(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You answer the user's question using the given tool outputs.\n"
                    "JSON contains user_question, tool_calls, and tool_results (already truncated).\n"
                    "1. Answer clearly in markdown.\n"
                    "2. At the end, add:\n"
                    "## Tools used\n"
                    "- One bullet per distinct tool name.\n"
                ),
            },
            {
                "role": "user",
                "content": json.dumps(payload, ensure_ascii=False),
            },
        ],
        temperature=0,
        max_tokens=600,
    )

    msg = resp.choices[0].message
    parts = getattr(msg, "content", None)
    if isinstance(parts, list):
        texts = []
        for p in parts:
            text = getattr(p, "text", None) or getattr(p, "content", None)
            if isinstance(text, str):
                texts.append(text)
        if texts:
            return "".join(texts)

    return str(msg.content)


## Orchestrate (connect both servers and execute tool_calls)

In [13]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def orchestrate(prompt: str):
    local_params = StdioServerParameters(
        command="mcp",
        args=["run", str(LOCAL_SERVER)],
        env=BASE_ENV,  # <- use merged env
    )

    if USE_REAL_AIRBNB:
        airbnb_params = StdioServerParameters(
            command="npx",
            args=["@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
            env=BASE_ENV,
        )


    async with stdio_client(local_params) as (lr, lw):
        async with ClientSession(lr, lw) as local_sess:
            await local_sess.initialize()
            local_tools = await local_sess.list_tools()

            async with stdio_client(airbnb_params) as (ar, aw):
                async with ClientSession(ar, aw) as airbnb_sess:
                    await airbnb_sess.initialize()
                    airbnb_tools = await airbnb_sess.list_tools()

                    functions = (
                        [convert_tool(t, "notes") for t in local_tools.tools]
                        + [convert_tool(t, "airbnb") for t in airbnb_tools.tools]
                    )

                    tool_calls = call_llm(prompt, functions, use_real=USE_REAL_LLM)
                    print("tool_calls:", tool_calls)

                    tool_results = []
                    for call in tool_calls:
                        name = call["name"]
                        args = call["args"]
                        prefix, tool_name = name.split("__", 1)

                        if prefix == "notes":
                            res = await local_sess.call_tool(tool_name, args)
                            tool_results.append(
                                {
                                    "name": name,
                                    "args": args,
                                    "content": [c.text for c in res.content if hasattr(c, "text")],
                                }
                            )
                        elif prefix == "airbnb":
                            res = await airbnb_sess.call_tool(tool_name, args)
                            payload = []
                            if hasattr(res, "content"):
                                for c in res.content:
                                    if hasattr(c, "text"):
                                        payload.append(c.text)
                            tool_results.append(
                                {
                                    "name": name,
                                    "args": args,
                                    "content": payload,
                                }
                            )

                    # note: DO NOT call answer_with_llm here if you want to inspect things first
                    return functions, tool_calls, tool_results

## Demo
Adjust the prompt as you like. Switch `USE_REAL_AIRBNB/USE_REAL_LLM` to true when ready.

In [14]:
prompt = "What tools can you access? List them please."

# Call orchestrate and display results
functions_available, llm_tool_calls, tool_results = await orchestrate(prompt)

print("=== AVAILABLE TOOLS ===")
for t_dict in functions_available:
    fn_info = t_dict['function']
    print(f"- {fn_info['name']}: {fn_info['description']}")

print("\n=== LLM TOOL CALLS ===")
for call in llm_tool_calls:
    print(call)


print("\n=== TOOL RESULTS ===")
for result in tool_results:
    print(result)

tool_calls: []
=== AVAILABLE TOOLS ===
- notes__add_note: Add a note to the in-memory list.
- notes__list_notes: List saved notes.
- airbnb__airbnb_search: Search for Airbnb listings with various filters and pagination. Provide direct links to the user
- airbnb__airbnb_listing_details: Get detailed information about a specific Airbnb listing. Provide direct links to the user

=== LLM TOOL CALLS ===

=== TOOL RESULTS ===


In [17]:
prompt = "What tools can you access ? list them please "

functions_available, llm_tool_calls, tool_results = await orchestrate(prompt)

print("=== AVAILABLE TOOLS ===")
for t_dict in functions_available:
    fn_info = t_dict['function']
    print(f"- {fn_info['name']}: {fn_info['description']}")

print("\n=== LLM TOOL CALLS ===")
for call in llm_tool_calls:
    print(call)


print("\n=== TOOL RESULTS ===")
for result in tool_results:
    print(result)

tool_calls: []
=== AVAILABLE TOOLS ===
- notes__add_note: Add a note to the in-memory list.
- notes__list_notes: List saved notes.
- airbnb__airbnb_search: Search for Airbnb listings with various filters and pagination. Provide direct links to the user
- airbnb__airbnb_listing_details: Get detailed information about a specific Airbnb listing. Provide direct links to the user

=== LLM TOOL CALLS ===

=== TOOL RESULTS ===
